## This notebook aims to develope the Accuracy testing

In [1]:
import numpy as np 
from tools.auxiliary import*
from tools.GH_Quadrature import*
from tools.Ord_Herm_Pol_1 import*
from tools.Num_Stab_Approx import*
from tools.Main_Results import*
from tools.Accuracy_Test_1 import*
import pandas as pd
import math
import time
from statistics import*

In [2]:
df =reading("epsi10000.csv")
T  = 10000           #Choose the simulation length for the solution procedure, T<=10,000                    
gam     = 1        # Utility-function parameter
alpha   = 0.36     # Capital share in output
beta    = 0.99     # Discount factor
delta   = 0.02     # Depreciation rate 
rho     = 0.95     # Persistence of the log of the productivity level
sigma   = 0.01    # Standard deviation of shocks to the log of the productivity level
ks = ( (1-beta+beta*delta) / (alpha*beta) )**(1/(alpha-1) )
k = np.array([ks]*(T+1))
a= [1]*(T)
epsi = df.iloc[:,0].astype(float)*sigma
epsi=epsi.tolist()
for i in range(1, T):
    a[i]=a[i-1]**(rho)*math.exp(epsi[i])
a=np.asarray(a)
kdamp = 0.01    
dif_GSSA_1d = 1e+10  
bk_1d  = np.array([0., 0.95, ks*0.05])
bk_1d= np.reshape(bk_1d, (3,1))
k_old = [ks+1]*(T+1)

In [3]:
start = time.time()
y= GSSA_main_cycle(T, gam, alpha, beta, delta, kdamp, dif_GSSA_1d, a, bk_1d, k_old, k)
end = time.time()
elapsed_time = end-start

In [4]:
y = y.reshape((y.shape[0],1)) #make sure y is in the right shape
#The GSSA parameters

kdamp = 0.1
dif_GSSA_D = 1e+10

#The matrices of the polynomial coefficients
D_max  = 5 #because of python
npol = np.array([3, 6, 10, 15, 21])

# 13. Choose an integration method for computing solutions  
IM  = 10

n_nodes,epsi_nodes, weight_nodes= GH_Quadrature(Qn=10, N=1, vcv=sigma**2)

#make sure to change a into the right shape
a = np.reshape(a, (T, 1))
a1 = np.matmul(np.power(a,rho), np.exp(epsi_nodes.transpose()))

#14. Choose a regression specification 
RM = 6           # Choose a regression method: 
                 # 1=OLS,          2=LS-SVD,   3=LAD-PP,  4=LAD-DP, 
                 # 5=RLS-Tikhonov, 6=RLS-TSVD, 7=RLAD-PP, 8=RLAD-DP
normalize = 1    # Option of normalizing the data; 0=unnormalized data; 
                 # 1=normalized data                    
penalty = 7      # Degree of regularization for a regularization methods, 
                 # RM=5,6,7,8 (must be negative, e.g., -7 for RM=5,7,8 
                 # and must be positive, e.g., 7, for RM=6)
PF = 0           # Choose a polynomial family; 0=Ordinary (default);  
                 # 1=Hermite
# 15. Initialize the capital series
zb = np.matrix([[np.mean(k[0:T]), np.mean(a[0:T])], [np.std(k[0:T]), np.std(a[0:T])]])
z = np.concatenate((k[0:T].reshape(T,1), a[0:T].reshape(T,1)), axis=1)
k_old = [ks+1]*(T+1)

In [5]:
BK = []
Time = []
for d in range(1, D_max+1):
    start = time.time()
    BK.append(GSSA_poly(T, a, z, d, PF, zb, RM, penalty, normalize, dif_GSSA_D, kdamp, alpha, beta, delta, k, gam, y, k_old, a1, IM, n_nodes, weight_nodes, checker= 0))
    end = time.time()
    Time.append(end-start)

In [25]:
BK[0]

array([[-1.76699623],
       [ 0.97049037],
       [ 3.19294961]])

In [27]:
BK[0].shape

(3, 1)

## Accuracy:

In [10]:
T_test = 10200

df =reading("epsi_test.csv")
epsi_test = sigma*df.to_numpy().astype(float)
a_test = [1]
for i in range(1,T_test):
    value = a_test[i-1]**(rho)*math.exp(float(epsi_test[i]))
    a_test.append(value)

IM_test = 10

k_test = [ks]
result_max = []
result_mean = []
result_time = []
for d in range(1, D_max+1):
    #refressing k_test to make sure that k_test is always 10200
    #k_test = [ks]
    for i in range(T_test):
        X_test = Ord_Herm_Pol_1(np.array([k_test[i], a_test[i]]).reshape([1,2]),d,PF,zb) # D = 1 for now, we will plug this in another for loop
        value = float(np.matmul(X_test, BK[d-1]))
        k_test.append(value)

        # testing it below
    discard = 200 #new defined value
    mean_error, max_error, error_time = Accuracy_Test_1(sigma,rho,beta,gam,alpha,delta,k_test,a_test,BK[d-1],d,IM_test,PF,zb,discard)
    result_max.append(max_error)
    result_mean.append(mean_error)
    result_time.append(error_time)

In [11]:
for d in range(1, D_max+1):
    #refressing k_test to make sure that k_test is always 10200
    #k_test = [ks]
    for i in range(T_test):
        X_test = Ord_Herm_Pol_1(np.array([k_test[i], a_test[i]]).reshape([1,2]),d,PF,zb) # D = 1 for now, we will plug this in another for loop
        value = float(np.matmul(X_test, BK[d-1]))
        k_test.append(value)

        # testing it below
    discard = 200 #new defined value
    mean_error, max_error, error_time = Accuracy_Test_1(sigma,rho,beta,gam,alpha,delta,k_test,a_test,BK[d-1],d,IM_test,PF,zb,discard)
    result_max.append(max_error)
    result_mean.append(mean_error)
    result_time.append(error_time)


In [7]:
d = 1
for i in range(T_test):
    X_test = Ord_Herm_Pol_1(np.array([k_test[i], a_test[i]]).reshape([1,2]),d,PF,zb) # D = 1 for now, we will plug this in another for loop
    value = float(np.matmul(X_test, BK[d-1]))
    k_test.append(value)

        # testing it below


In [8]:
discard = 200 #new defined value
mean_error, max_error, error_time = Accuracy_Test_1(sigma,rho,beta,gam,alpha,delta,k_test,a_test,BK[d-1],d,IM_test,PF,zb,discard)

In [42]:
test=[]
k0 = float(k[0])
a0 = float(a[0])
k1 = float(Ord_Herm_Pol_1(np.array([k0,a0]).reshape((1,2)), d, PF=PF, zb=zb)@BK[0])
c0 = k0**alpha*a0 - k1+ (1-delta)*k0
a1 = a0**rho*np.exp(epsi_nodes)
k1_dupl1 = k1*np.ones((n_nodes,1))
x1 = Ord_Herm_Pol_1(np.concatenate((k1_dupl1, a1), axis=1),d,PF,zb)
k2 = x1@BK[0]
c1 = k1**alpha*a1 - k2 + (1-delta)*k1
test.append(weight_nodes.conj().transpose()@(beta*c1**(-gam)/c0**(-gam)*(1-delta+alpha*a1*k1**(alpha-1))) -1)



Prerequisite:

Testing area:

In [48]:
def Accuracy_Test_1(sigma,rho,beta,gam,alpha,delta,k,a,bk,D,IM,PF,zb,discard):
    '''
    '''
    start = time.time()
    n_nodes,epsi_nodes, weight_nodes = GH_Quadrature(Qn=IM, N=1, vcv=sigma**2)
    #initialise empty list
    errors = []
    for i in range(len(a)):
        k0 = float(k[i])
        a0 = float(a[i])
        k1 = float(Ord_Herm_Pol_1(np.array([k0,a0]).reshape((1,2)), D, PF=PF, zb=zb)@bk)
        c0 = k0**alpha*a0 - k1+ (1-delta)*k0
        a1 = a0**rho*np.exp(epsi_nodes)
        k1_dupl1 = k1*np.ones((n_nodes,1))
        x1 = Ord_Herm_Pol_1(np.concatenate((k1_dupl1, a1), axis=1),D,PF,zb)
        k2 = np.matmul(x1, bk)
        c1 = k1**alpha*a1 - k2 + (1-delta)*k1
        errors.append(weight_nodes.conj().transpose()@(beta*c1**(-gam)/c0**(-gam)*(1-delta+alpha*a1*k1**(alpha-1))) -1) #testing new approach
    Error_means = math.log10(np.mean(np.mean(np.abs(errors[discard:]))))
    Error_max = math.log10(np.max(np.max(np.abs(errors[discard:]))))
    end = time.time()
    time_test = end - start
    return Error_means, Error_max, time_test

In [11]:
errors = []
for i in range(a):
    k0 = float(k[i])
    a0 = float(a[i,0])
    k1 = float(np.matmul(Ord_Herm_Pol_1(np.array([k0,a0]).reshape((1,2)), D, PF=PF, zb=zb), bk))
    c0 = k0**alpha*a0 - k1+ (1-delta)*k0
    a1 = a0**rho*np.exp(epsi_nodes)
    k1_dup1 = k1*np.ones((n_nodes,1))
    x1 = Ord_Herm_Pol_1(np.concatenate((k1_dupl1, a1), axis=1),D,PF,zb)
    k2 = np.matmul(x1, bk)
    c1 = k1**alpha*a1 - k2 + (1-delta)*k1
    errors.append(weight_nodes.conj().transpose()@(beta*c1**(-gam)/c0**(-gam)*(1-delta+alpha@a1*k1**(alpha-1))) ) #testing new approach
Error_means = math.log10(mean(mean(abs(errors[discard:]))))
Error_max = math.log10(max(max(abs(errors[discard:]))))


1.0

In [33]:
    i = 1
    k0 = float(k[i])
    a0 = float(a[i,0])
    k1 = float(np.matmul(Ord_Herm_Pol_1(np.array([k0,a0]).reshape((1,2)), D=1, PF=PF, zb=zb), BK[0]))
    c0 = k0**alpha*a0 - k1+ (1-delta)*k0
    a1 = a0**rho*np.exp(epsi_nodes)
    k1_dupl1 = k1*np.ones((n_nodes,1))
    x1 = Ord_Herm_Pol_1(np.concatenate((k1_dupl1, a1), axis=1), 1, PF, zb)
    k2 = np.matmul(x1, BK[0])
    c1 = k1**alpha*a1 - k2 + (1-delta)*k1
    errors = weight_nodes.conj().transpose()@(beta*c1**(-gam)/c0**(-gam)*(1-delta+alpha*a1*k1**(alpha-1))) -1 #testing new approach



In [31]:
beta*c1**(-gam)/c0**(-gam)*(1-delta+alpha*a1*k1**(alpha-1))

array([[0.98793406],
       [0.99112902],
       [0.99385936],
       [0.99638101],
       [0.9987996 ],
       [1.0011802 ],
       [1.00357767],
       [1.00605495],
       [1.00871185],
       [1.01178787]])

In [22]:
weight_nodes'*(beta*c1(1:n_nodes,1).^(-gam)./c0(1,1).^(-gam).*(1-delta+alpha*a1(1:n_nodes,1).*k1(1,1).^(alpha-1)))-1;

[4, 5]